In [32]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools
using StaticArrays

using Profile
using PProf
using ProfileSVG

import Base: show

In [33]:
snapshot(var) = var.belief.subject.buffer[1]

snapshot (generic function with 1 method)

In [40]:
meanv = datavar(:mean, NormalMeanPrecision{Float64})
precv = datavar(:precision, GammaAB{Float64})

gmpnode = GaussianMeanPrecisionNode(factorisation = SA[ SA[1], SA[2], SA[3] ])

connect!(gmpnode, :mean, meanv, 1)
connect!(gmpnode, :precision, precv, 1)

y = datavar(:y, Float64)

connect!(gmpnode, :value, y, 1)

activate!(gmpnode)

# qmean = subscribe!(getbelief(meanv) |> map(Any, getdata), logger("qmean"))
# qprec = subscribe!(getbelief(precv) |> map(Any, getdata), logger("qprec"))
# qy    = subscribe!(getbelief(y)     |> map(Any, getdata), logger("qy   "))

qmean = subscribe!(getbelief(meanv) |> map(Any, getdata), void())
qprec = subscribe!(getbelief(precv) |> map(Any, getdata), void())
qy    = subscribe!(getbelief(y)     |> map(Any, getdata), void())

# mmean = subscribe!(messagein(meanv, 1), logger("m_mean"))
# mprec = subscribe!(messagein(precv, 1), logger("m_prec"))
# my    = subscribe!(messagein(y, 1),     logger("m_y   "))

setbelief!(meanv, Belief(NormalMeanPrecision(0.0, 0.001)))
setbelief!(precv, Belief(GammaAB(0.001, 0.001)))

update!(meanv, NormalMeanPrecision(0.0, 0.001))
update!(precv, GammaAB(0.001, 0.001))

In [41]:
realprecision = 3.0
realmean = -10

data = rand(Normal(realmean, sqrt(1.0 / realprecision)), (1000, ));

In [42]:
current_mean_prior = NormalMeanPrecision(0.0, 0.001)
current_prec_prior = GammaAB(0.001, 0.001)
update!(precv, current_prec_prior)
update!(meanv, current_mean_prior)

for i in 1:1000
    for j in 1:50
        update!(y, data[i])
    end
    current_mean_prior = getdata(snapshot(meanv))
    current_prec_prior = getdata(snapshot(precv))
    update!(precv, current_prec_prior)
    update!(meanv, current_mean_prior)
end

println("mean: ", Distributions.mean(current_mean_prior))
println("prec: ", Distributions.mean(current_prec_prior))

mean: -9.974673084702825
prec: 2.791032737420971


In [61]:
current_mean_prior

NormalMeanPrecision{Float64}(0.024892099062898514, 40596.26681099683)

In [223]:
Message(NormalMeanPrecision(0.0, 0.001)) * Message(NormalMeanPrecision(10.0, 0.001))

Message{NormalMeanPrecision{Float64}}(NormalMeanPrecision{Float64}(5.0, 0.002))

In [226]:
Message(GammaAB(0.001, 0.001)) * Message(GammaAB(0.001, 0.001))

Message{GammaAB{Float64}}(GammaAB{Float64}(-0.998, 0.002))